In [128]:
import openai
from openai.types import Completion, CompletionChoice, CompletionUsage
import os
import requests
from dotenv import load_dotenv
from scipy.optimize import minimize, Bounds, NonlinearConstraint
from functools import partial 

load_dotenv("/Users/gabriellelittlefair/Documents/GitHub/MSc Project/DSL/hidden.env")
%reload_ext autoreload
from Setup_Functions import *
from Class_Structures import *
from Region import * 
from Individual import * 
from Pairwise import * 

scene_descriptor = "a bedroom for a couple"

# Set up the Room (all dimensions + fixed features)

In [129]:
api_key = os.getenv('OPENAI_API_KEY')
client = openai.Client(api_key=api_key)
# If you're setting the API key directly in your script, uncomment the next line 


def remove_identical_lines(function):
    lines = function.split("\n")
    unique_lines = []
    for line in lines: 
        if line not in unique_lines:
            unique_lines.append(line)
    return("\n").join(unique_lines)


def room_setup(scene_descriptor, api_key=api_key):
    url = 'https://api.openai.com/v1/chat/completions'
    headers = {
        'Authorization': f'Bearer {api_key}', 
        'Content-Type': 'application/json',
            }
    
    # Read the contents of the .py file
    file_path = '/Users/gabriellelittlefair/Documents/GitHub/MSc Project/DSL/Setup_Functions.py'
    with open(file_path, 'r') as file:
        file_contents = file.read()

    prompt1 = "I am designing the layout for" + scene_descriptor + ". Please give me the dimensions of the room,"
    prompt1 += """ as well as the number of windows, doors, and sockets. For each window, door, and socket tell me the dimensions (in meters),
    which wall it should be on, as well as where on the wall it should be (this is a value between 0 and 1 determining how far along the wall it should be). Here is an example for a room and window set up:
    {Room: width = 5,  length = 6}, {Object type: window, width = 1, length = 0.1, wall = south, position = 3}, etc.
    The windows and doors should all have length 0.1. Ensure that every object has dimensions, position, and wall.
     No other text please. Just the list with no commentary or explanation."""
    response1 = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt1}],
        max_tokens=500
    )
    objects = response1.choices[0].message.content.split("}, {")
    prompt2 = "I am going to give you a set of functions that are used to create a room and put objects in it. The functions are as follows:"
    prompt2 += file_contents
    prompt2 += f"I now want, for each element in this list: {objects}, with room descriptor:" + scene_descriptor + ", for you to translate it into a function call using the functions I gave you above."
    prompt2 += " For example, given the room descriptor 'a small kitchen' as well as the list: "
    prompt2 += "{Room: width = 5,  length = 6}, {Object type: window, Number: 1,  width = 1, length = 0.1, wall = south, position = 0.3}, the output would be:"
    prompt2 += "kitchen = create_room(5, 6), create_fixed_object(kitchen, 'window', 1, 0.1, 'south', position = 0.3)."
    prompt2 += " Return only the python script. No other text. Please ensure the correct number of inputs are given for each function."

    # Define the request payload
    data = {
        "model": "gpt-4",
        "messages": [
            {"role": "user", "content": prompt2}
        ]
    }
    
    # Make the API call
    response2 = requests.post(url, headers=headers, json=data)  
    # Check the response
    if response2.status_code == 200:
        response_data2 = response2.json()
        if "```python" in response_data2['choices'][0]['message']['content']:
            response2 = (response_data2['choices'][0]['message']['content'].split("```python")[1]).split("```")[0]
        elif "```" in response_data2['choices'][0]['message']['content']:
            response2 = (response_data2['choices'][0]['message']['content'].split("```")[1]).split("```")[0]
        else: 
            response2 = response_data2['choices'][0]['message']['content']
    else:
        print('Failed to get a response')
        print(response2.text)

    return response1, response2

response1, response2 = room_setup(scene_descriptor)

## Check that the room was created, if not, rerun the room setup function
if 'create_room' not in response2: 
    print("response2 did not setup the room correctly, rerunning the room setup.")
    response1, response2 = room_setup(scene_descriptor)

## Account for any additional function calls (of other functions) by only taking the lines in the code that are relevant to the task   
lines = response2.split("\n")
response2 = [i for i in lines if 'create_room' in i]
for line in lines[1:]: 
    if "create_fixed_object" in line: 
        response2.append(line)

response2 = ("\n").join(response2)
width = (response2.split("create_room("))[1].split(",")[0]
length = (response2.split("create_room("))[1].split(",")[1].split(")")[0]
room_name = (response2.split("create_room("))[0].split("=")[0].strip()

lang_prompt1 = "I am designing the layout of a" +  scene_descriptor + "that is" + width + "m x " + length + "m."
lang_prompt1 += """Interior designers suggest that a good layout will have different regions in the room for different functions.
What regions should I use, and how many regions should there be based on the description (minimum is 1 region and maximum is 5) """ + scene_descriptor + """, as well 
as the size of the room (if the room is small, don't give too many regions). Some rooms may only need one region and thats okay. Return this as a simple bullet list of regions to include, if there are choices to be made e.g. which of the regions
or how many regions, please make those choices. Return these in list of priority order. Only return the list of regions and the number of regions. No other information please."""

lang1 = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": lang_prompt1}],
        max_tokens=500
    )

print(lang1.choices[0].message.content)

url = 'https://api.openai.com/v1/chat/completions'
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
        }

# Read the contents of the .py file
file_path = '/Users/gabriellelittlefair/Documents/GitHub/MSc Project/DSL/Setup_Functions.py'
with open(file_path, 'r') as file:
    file_contents = file.read()

prompt3 = "Given the room name: " + room_name + ", and the regions: " + lang1.choices[0].message.content
prompt3 += " Using the functions in the script, initialise each region. For example 'eating region' might output 'eating_region = region_setup(kitchen, 'eating', 0)."
prompt3 += " No other text please, only the python script. Don't include the room so far in the output."
prompt3 += file_contents
 # Define the request payload
data = {
    "model": "gpt-4",
    "messages": [
            {"role": "user", "content": prompt3}
            ]
    }
    
# Make the API call
response3 = requests.post(url, headers=headers, json=data)  
# Check the response
if response3.status_code == 200:
    response_data3 = response3.json()
    if "```python" in response_data3['choices'][0]['message']['content']:
        ## Remove any function calls that aren't region setups
        response3 = response_data3['choices'][0]['message']['content'].split("```python")[1].split("```")[0]
        lines = response3.split("\n")
        response3 = []
        for line in lines: 
            if "region_setup" in line: 
                response3.append(line)
        response3 = ("\n").join(response3)
        response3 = response2 + "\n" + (response_data3['choices'][0]['message']['content'].split("```python")[1]).split("```")[0]
    else: 
        ## Remove any function calls that aren't region setups
        lines = response_data3['choices'][0]['message']['content'].split("\n")
        response3 = []
        for line in lines: 
            if "region_setup" in line: 
                response3.append(line)
        response3 = ("\n").join(response3)
        response3 = response2 + "\n" + response_data3['choices'][0]['message']['content']
        global_context = globals().copy()
        local_context = {}  
        exec(response3, global_context, local_context)
else:
    print('Failed to get a response')
    print(response3.text)
print(response3)

fixed_objects = ""
for i in range(len(local_context[room_name].fixed_objects)):
    if i < len(local_context[room_name].fixed_objects) - 1:
        fixed_objects +=local_context[room_name].fixed_objects[i].name + ", "
    else: 
        fixed_objects += local_context[room_name].fixed_objects[i].name + "."

room_width = local_context[room_name].width
room_length = local_context[room_name].length

region_names = ""
list_region_names = []
for i in range(len(local_context[room_name].regions)):
    if i < len(local_context[room_name].regions) - 1:
        region_names += local_context[room_name].regions[i].name + ", "
        list_region_names.append(local_context[room_name].regions[i].name)  
    else: 
        region_names += local_context[room_name].regions[i].name + "."
        list_region_names.append(local_context[room_name].regions[i].name)
num_primary_objects = len(list_region_names)
num_regions = len(list_region_names)

- Sleeping Area
- Storage/Closet
- Dressing Area
room = create_room(4, 5)
create_fixed_object(room, 'window', 1.2, 0.1, 'north', position = 0.3)
create_fixed_object(room, 'window', 1.2, 0.1, 'south', position = 0.7)
create_fixed_object(room, 'door', 0.9, 0.1, 'east', position = 0.1)
create_fixed_object(room, 'socket', 0.15, 0.15, 'west', position = 0.4)
create_fixed_object(room, 'socket', 0.15, 0.15, 'east', position = 0.8)
create_fixed_object(room, 'socket', 0.15, 0.15, 'north', position = 0.6)
sleeping_area = region_setup(room, 'Sleeping Area', 0)
storage_closet = region_setup(room, 'Storage/Closet', 1)
dressing_area = region_setup(room, 'Dressing Area', 2)
['Sleeping Area', 'Storage/Closet', 'Dressing Area']


# Get the primary objects, as well as all of their constraints. 

In [130]:
lang_prompt2 = """The room is described as """ + scene_descriptor + """. Now for each one of the regions returned in this:""" + lang1.choices[0].message.content + """, what
is the most important object to include (per region), and what are its dimensions (give width of back of object and length of side of object not height) 
all in meters. ONLY one object per region (there can be two of the same object if NECESSARY if that is the case, output them as bed 1 and bed 2 or nighstand 1 and nightstand 2). The size of the room is"""  + width + "m x " + length + """m, bear this in mind when choosing the objects and the 
size of the objects. Give no other information please. The primary objects should not cover more than 20 percent of the room area."""


lang2 = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": lang_prompt2}],
        max_tokens=2000
    )

print(lang2.choices[0].message.content)

lang_prompt3 = """For each of these objects""" + lang2.choices[0].message.content + """, tell me all of the considerations and constraints for its placement 
within the room that depends only on fixed features in the room like walls, windows, doors, sockets, etc... 
(return these as a bullet list for each object). Include practical things like whether it should be against a wall 
and what side (one of: left, right, top/back, bottom/front) of it should be against a wall (don't describe this as long or short side, if its something like a headboard or the back of a sofa that would be 'back' etc.), or which side should be accessible for use etc. 
Only give these constraints and considerations, no other information. """
lang3 = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": lang_prompt3}],
        max_tokens=2000
    )

print(lang3.choices[0].message.content)

lang_prompt4 = """ Tell me all of the constraints and considerations between the objects in this list""" + lang3.choices[0].message.content + """ that depend only on each other.
For example, maybe a desk should not be close to a bed, etc. Only give the constraints and considerations between objects, no other information.."""

lang4 = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": lang_prompt4}],
        max_tokens=2000
    )

print(lang4.choices[0].message.content)

- Sleeping Area: Bed - Width: 1.8m, Length: 2.0m
- Storage/Closet: Wardrobe - Width: 1.5m, Length: 0.6m
- Dressing Area: Dresser - Width: 1.2m, Length: 0.5m
### Bed 
- Should be placed against a wall for stability and space efficiency.
- Side considerations: 
  - Headboard (top) should be against a wall.
  - At least one long side should be accessible for easy entry and exit.
- Avoid placement directly under windows due to drafts and sunlight.
- Keep away from doors to ensure clear access and avoid obstructing pathways.
- Consider proximity to power outlets for bedside lamps or charging devices.

### Wardrobe
- Should be placed against a wall for stability and space efficiency.
- Side considerations:
  - Back should be against a wall.
  - Front should be totally accessible for door operation.
- Avoid placement near windows to prevent obstructing light and drafts.
- Position near dressing area for convenience.

### Dresser
- Should be placed against a wall for stability and space effici

# Get the Secondary objects, as well as all of their constraints

In [131]:
lang_prompt5 = "Here is a description of a room: " + scene_descriptor + ". These are the regions within the room: " + region_names + "."
lang_prompt5 += "These are the primary objects that are already within the room: " + str(primary_objects) + "."
lang_prompt5 += "And this is the size of the room: width = " + str(room_width) + " meters, length = " + str(room_length) + " meters."
lang_prompt5 += """ Give me 0-10 (depending on the size of the room) more objects that should be added into the room (so make sure they are the most appropriate/necessary objects), 
for each one, make sure that they make sense to go into one of the regions, and tell me which region they should go into. 
There can be more than one of each object kind (eg. multiple dining chairs or multiple nightstands). Make sure to also give the dimensions of every object
in meters (width (of the back of the object) and length (side of the object), NOT HEIGHT). Don't give any objects like table lamps that would go on top of other objects.
Give no other text in the response. Only the list of objects. Remove any objects that are not placed directly on the floor, e.g. table lamp."""

lang5 = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": lang_prompt5}],
        max_tokens=1000
    )

print(lang5.choices[0].message.content)

lang_prompt6 = """For each of these objects""" + lang5.choices[0].message.content + """, tell me all of the considerations and constraints for its placement 
within the room that depends only on fixed features in the room like walls, windows, doors, sockets, etc. (return these as a bullet list for each object). 
Include practical things like whether it should be against a wall and what side of the object (one of: left, right, top/back, bottom/front) of it should be
against a wall (don't describe this as long or short side, if its something like a headboard or the back of a sofa that would be 'back' etc.), 
or which side should be accessible for use etc. Don't include any constraints that depend on other objects. Only give these constraints and considerations, no other information. """
lang6_1 = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": lang_prompt6}],
        max_tokens=2000
    )

lang_prompt6 = """ For all of these constraints, remove any that related to other objects. I only want constraints 
that depend on fixed features in the room like walls, windows, doors, sockets, etc. Feel free to add any extra details like minimum distances etc to existing constraints.
Don't create any new constraints. I also want you to only enforce an object being against a wall if necessary. For example, 
the back of a bed MUST be against a wall, but the back of a desk chair does not need to be, nor does a sofa generally (sometimes it does).
If any constraints contain two options e.g 'the bed should be against the left wall or the right wall', change it to be only one option."""
lang_prompt6 += lang6_1.choices[0].message.content
lang6 = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": lang_prompt6}],
        max_tokens=2000
    )

print(lang6.choices[0].message.content)

lang_prompt7 = """ Tell me all of the constraints and considerations between the objects in this list""" + lang6.choices[0].message.content + """ as well as 
in this list """ + str(primary_objects) + """. For example, a desk chair should have its front against the front of the desk, or the left side of one 
of the nightstands should be against the right side of the bed, etc.  Be specific with relationships between objects to include sides (one of: left, right, top/back, bottom/front) 
if appropriate, or minimum/maximum distances between objects etc.
Only give the constraints and considerations between objects, no other information."""

lang7 = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": lang_prompt7}],
        max_tokens=2000
    )

lang_prompt8 = " Given the description of the room: " + scene_descriptor + " and this list of constraints: " + lang7.choices[0].message.content + "."
lang_prompt8 += """ Tidy the list by choosing between any options and removing any constraints that are repeated. Give any extra details to a constraint that might be necessary
For example the constraint might be 'One side (left or right) of the nightstand should be against the bed.' and you might change it to 
'The left side of the nightstand should be against the right side of the bed'. In that example, I chose between left and right for the nightstand, and then added 
the extra detail of what side of the bed it should be against. Also, only have objects against the wall if they need to be against the wall. If it is "can be against a wall" remove it.
For example, the back of a bed MUST be against the wall, but, a desk chair does not need to be, neither does a side table. If possible, simplify the language of each constraint 
as if to explain it to a child. Remember to remove any constraints from the list that are repeated. Only return the list 
of constraints and considerations. No other information."""

lang8 = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": lang_prompt8}],
        max_tokens=2000
    )

print(lang8.choices[0].message.content)

1. Nightstand - Sleeping Area - 0.5m x 0.4m
2. Nightstand - Sleeping Area - 0.5m x 0.4m
3. Dresser - Dressing Area - 1.0m x 0.5m
4. Full-length Mirror - Dressing Area - 0.6m x 0.4m
5. Rug - Sleeping Area - 2.0m x 1.5m
6. Bench - Dressing Area - 1.2m x 0.4m
7. Shoe Rack - Storage/Closet - 0.8m x 0.3m
8. Laundry Basket - Storage/Closet - 0.4m x 0.4m
Certainly! Below are the constraints for each object, considering only the fixed features in the room such as walls, windows, doors, and sockets. Any constraints involving other objects have been removed, and details like minimum distances have been kept or added where necessary:

1. **Nightstand 1 (Sleeping Area - 0.5m x 0.4m)**
   - Should be placed against a wall.
   - The back should be against the wall.
   - Should not block windows or doorways.
   - Electrical sockets nearby for lamp or charging devices.

2. **Nightstand 2 (Sleeping Area - 0.5m x 0.4m)**
   - Should be placed against a wall.
   - The back should be against the wall.
   

# Add all the objects into the room 
### Region by region. 

In [228]:
prompt4 = "Given this list of the objects: " + lang2.choices[0].message.content +  lang5.choices[0].message.content + ", use the file attached to create the objects with the correct parameters."
prompt4 += "The room name is: " + room_name + ", the region names are: " + region_names + "."
prompt4 += "The room is already set up, only add in the objects using the 'create_moving_object' function."
prompt4 += "The objects should be added in the correct regions. I want the objects to be added region by region (i.e. all objects from region 0 should be added before region 1 objects are added). "
prompt4 += "This is the file: " + file_contents
prompt4 += "No extra text, only the function calls. Don't have 'python' at the start of the code. Do not define ANY functions, only call them."

# Define the request payload
data = {
    "model": "gpt-4o",
    "messages": [
            {"role": "user", "content": prompt4}
            ]
    }
    
# Make the API call
response4 = requests.post(url, headers=headers, json=data)  
# Check the response
if response4.status_code == 200:
    response_data4 = response4.json()


    if "```" in response_data4['choices'][0]['message']['content']:
        list_objects = (response_data4['choices'][0]['message']['content']).split("\n")[1:-1]
        response4_1 = ""
        for i in range(len(list_objects)):
            response4_1 += list_objects[i] + "\n"
    else: 
        lines = response_data4['choices'][0]['message']['content'].split("\n")
        response4_1 = []
        for line in lines: 
            if "create_moving_object" in line: 
                response4_1.append(line)
        response4_1 = ("\n").join(response4_1)
    
    response4_1 = response4_1.replace("create_moving_object(room,", "create_moving_object(" + room_name + ",")
    response4 = remove_identical_lines(response4_1).replace(room_name, "local_context[room_name]")

else:
    print('Failed to get a response')
    print(response4.text)

strings = response4.split("create_moving_object(local_context[room_name],")
primary_objects = []
for string in strings[1:]:
    if "'" in string: 
        primary_objects.append(string.split("'")[1])
    else:
        primary_objects.append(string.split('"')[1])

print(response4)
object_areas = 0
room_area = room_width * room_length
object_sizes = response4.split("\n")
wl = []
for string in object_sizes: 
    if "create_moving_object" in string: 
        sub_strings = string.split(",")
        for sub_string in sub_strings: 
            if (sub_string.replace(" ", "")).replace(".", "").isnumeric():
                wl.append(float(sub_string.replace(" ", "")))

for i in range(len(wl)//2):
    object_areas += wl[2*i] * wl[2*i + 1]

percent_of_room = 100 * object_areas/room_area
print(percent_of_room)

create_moving_object(local_context[room_name], "bed", 1.8, 2.0, "Sleeping Area", 0)
create_moving_object(local_context[room_name], "nightstand", 0.5, 0.4, "Sleeping Area", 1)
create_moving_object(local_context[room_name], "nightstand", 0.5, 0.4, "Sleeping Area", 2)
create_moving_object(local_context[room_name], "rug", 2.0, 1.5, "Sleeping Area", 3)
create_moving_object(local_context[room_name], "wardrobe", 1.5, 0.6, "Storage/Closet", 4)
create_moving_object(local_context[room_name], "shoe rack", 0.8, 0.3, "Storage/Closet", 5)
create_moving_object(local_context[room_name], "laundry basket", 0.4, 0.4, "Storage/Closet", 6)
create_moving_object(local_context[room_name], "dresser", 1.2, 0.5, "Dressing Area", 7)
create_moving_object(local_context[room_name], "dresser", 1.0, 0.5, "Dressing Area", 8)
create_moving_object(local_context[room_name], "full-length mirror", 0.6, 0.4, "Dressing Area", 9)
create_moving_object(local_context[room_name], "bench", 1.2, 0.4, "Dressing Area", 10)

50.6000000

In [232]:
object_creations = ['' for i in range(2*num_regions)]
lines = response4.split("\n")
for i in range(num_regions):
    sub_lines = [line for line in lines if list_region_names[i] in line]
    object_creations[2*i] = sub_lines[0]
    object_creations[2*i + 1] = "\n".join(sub_lines[1:])

In [139]:
primary_objects = []
secondary_objects = []
primary_object_indices = []

for name in list_region_names:
    lines = [i for i in response4.split("\n") if name in i and 'create_moving_object' in i]
    if "'" in lines[0]: 
        name = lines[0].split("'")[1]
    else:
        name = lines[0].split('"')[1]
    index = int(lines[0].split(", ")[-1].split(")")[0])
    primary_object_indices += [index]
    primary_objects += [name]

    for line in lines[1:]:
        if "'" in line: 
            name = line.split("'")[1]
        else:
            name = line.split('"')[1]
        secondary_objects += [name]

secondary_object_indices = [i for i in range(len(secondary_objects) + len(primary_objects)) if i not in primary_object_indices]
print(str(primary_object_indices), str(primary_objects), str(secondary_object_indices), str(secondary_objects))

[0, 4, 7] ['bed', 'wardrobe', 'dresser'] [1, 2, 3, 5, 6, 8, 9, 10] ['nightstand', 'nightstand', 'rug', 'shoe rack', 'laundry basket', 'dresser', 'full-length mirror', 'bench']


In [94]:
file_path3 = '/Users/gabriellelittlefair/Documents/GitHub/MSc Project/BlankConstraints/Individual.py'
with open(file_path3, 'r') as file:
    file_contents3 = file.read()

file_path4 = '/Users/gabriellelittlefair/Documents/GitHub/MSc Project/BlankConstraints/Pairwise.py'
with open(file_path4, 'r') as file:
    file_contents4 = file.read()


prompt5 = "Given the room name: " + room_name + ", as well as the primary objects: " + str(primary_objects)
prompt5 += ", as well as their indices " + str(primary_object_indices) + ", and these constraints: " + lang3.choices[0].message.content
prompt5 += """ . Using the script attached, transform each constraint into function calls that will ensure that the primary objects are set up correctly. 
Match each constraint to the closest function in the script file by using the docstrings. If you cannot find a function that matches the constraint,
leave it out. Here is the first script: """
prompt5 += file_contents3
prompt5 += "For the second script, I want you to transform the pairwise constraints into function calls. Here is the second script: " + file_contents4
prompt5 += " and here is the list of pairwise constraints: " + lang4.choices[0].message.content
prompt5 += """. I want the output to begin with: def optimize_primary_objects(positions, room): \n output = 0, 
followed by each constraint function call added to the output, and then the output returned. Ensure to include the 
functions ind_in_bounds, and ind_no_overlap. No extra text please, only the functions and the output.
 Do not define ANY functions, only call them. ENSURE that all parameters match docstrings."""

# Define the request payload
data = {
    "model": "gpt-4o",
    "messages": [
            {"role": "user", "content": prompt5}
            ]
    }
    
# Make the API call
response5 = requests.post(url, headers=headers, json=data)  
# Check the response
if response5.status_code == 200:
    response_data5 = response5.json()

    if "```" in response_data5['choices'][0]['message']['content']:
        list_objects = (response_data5['choices'][0]['message']['content']).split("\n")[1:-1]
        response5_1 = ""
        for i in range(len(list_objects)):
            response5_1 += list_objects[i] + "\n"
    else: 
        
        response5_1 = response_data5['choices'][0]['message']['content']

    response5 = remove_identical_lines(response5_1)
    lines = response5.split("\n")
    for line in lines: 
        if "ind_in_region" in line: 
            lines.remove(line)
    response5 = ("\n").join(lines)
    print(response5)
    global_context = globals().copy()
    local_context = {}  
    exec(response5, global_context, local_context)
else:
    print('Failed to get a response')
    print(response5.text)


def optimize_primary_objects(positions, room):
    output = 0
    output += ind_next_to_wall(positions, room, 0, 'back')
    output += ind_away_from_fixed_object(positions, room, 0, 'window')
    output += ind_away_from_fixed_object(positions, room, 0, 'door')
    output += ind_close_to_fixed_object(positions, room, 0, 'socket')
    output += ind_accessible(positions, room, 0, ['left', 'right'])
    output += ind_away_from_fixed_object(positions, room, 0, 'wardrobe')

    output += ind_next_to_wall(positions, room, 4, 'back')
    output += ind_away_from_fixed_object(positions, room, 4, 'bed')
    output += ind_accessible(positions, room, 4, ['front'])
    output += ind_not_block_fixed_object(positions, room, 4, 'window')
    output += ind_not_block_fixed_object(positions, room, 4, 'socket')
    output += ind_next_to_wall(positions, room, 8, 'back')
    output += ind_not_block_fixed_object(positions, room, 8, 'window')
    output += ind_away_from_fixed_object(positions, room, 8, 'door')

# Find the Primary Optimization Functions

In [244]:
primary_functions = []
lines = response5.split("\n")
for i in primary_object_indices[::-1]:#range(num_primary_objects - 1, -1, -1):
    print(i)
    lines_to_remove = []
    ## I need to split the optimization function into multiple different functons
    function = ("\n").join(lines[:2])
    for line in lines:
        sub_sections = line.split(room_name + ", ")
        if "ind_no_overlap" in line:
            function += "\n" + line
        if "ind_in_bounds" in line:
            function += "\n" + line
        if "return" in line: 
            function += "\n" + line
        for j in sub_sections: 
            if str(i) + "," in j or str(i) + ")" in j:
                function += "\n" + line
                lines_to_remove += [line]
                break 
    for line in lines_to_remove:
        lines.remove(line)
    print(function)
    primary_functions = [function] + primary_functions

7
def optimize_primary_objects(positions, room):
    output = 0
    output += ind_in_bounds(positions, room)
    output += ind_no_overlap(positions, room)
    return output
4
def optimize_primary_objects(positions, room):
    output = 0
    output += ind_next_to_wall(positions, room, 4, 'back')
    output += ind_away_from_fixed_object(positions, room, 4, 'bed')
    output += ind_accessible(positions, room, 4, ['front'])
    output += ind_not_block_fixed_object(positions, room, 4, 'window')
    output += ind_not_block_fixed_object(positions, room, 4, 'socket')
    output += p_away_from(positions, room, 0, 4)
    output += p_away_from(positions, room, 4, 8)
    output += p_aligned(positions, room, 0, 4, None, 2.0)
    output += p_aligned(positions, room, 4, 8, None, 2.0)
    output += ind_in_bounds(positions, room)
    output += ind_no_overlap(positions, room)
    return output
0
def optimize_primary_objects(positions, room):
    output = 0
    output += ind_next_to_wall(positions, room,

In [242]:
for i in primary_functions: 
    print(i)

def optimize_primary_objects(positions, room):
    output = 0
    output += ind_next_to_wall(positions, room, 0, 'back')
    output += ind_away_from_fixed_object(positions, room, 0, 'window')
    output += ind_away_from_fixed_object(positions, room, 0, 'door')
    output += ind_close_to_fixed_object(positions, room, 0, 'socket')
    output += ind_accessible(positions, room, 0, ['left', 'right'])
    output += ind_away_from_fixed_object(positions, room, 0, 'wardrobe')
    output += p_away_from(positions, room, 0, 8)
    output += p_aligned(positions, room, 0, 8, None, 2.0)
    output += ind_in_bounds(positions, room)
    output += ind_no_overlap(positions, room)
    return output
def optimize_primary_objects(positions, room):
    output = 0
    output += ind_next_to_wall(positions, room, 4, 'back')
    output += ind_away_from_fixed_object(positions, room, 4, 'bed')
    output += ind_accessible(positions, room, 4, ['front'])
    output += ind_not_block_fixed_object(positions, room, 4, 'w

In [182]:
primary_accessible_constraints = [[] for i in range(num_primary_objects)]
for obj_index in range(num_primary_objects):
    for i in primary_functions[obj_index].split("\n"):
        if "ind_accessible" in i:# and str(primary_object_indices[obj_index]) in i: 
            primary_accessible_constraints[obj_index].append(i)

    primary_accessible_constraints[obj_index] = "\n" +("\n").join(primary_accessible_constraints[obj_index])

print(primary_accessible_constraints[0])
print(primary_accessible_constraints[1])
print(primary_accessible_constraints[2])


    output += ind_accessible(positions, room, 0, ['left', 'right'])

    output += ind_accessible(positions, room, 4, ['front'])




# Get the Secondary Optimization Functions

In [169]:
objects_per_region =[[] for i in range(num_regions)]
for i in range(len(primary_objects)):
    if i < len(primary_objects) - 1:
        objects_per_region[i] = [j for j in range(primary_object_indices[i], primary_object_indices[i + 1])]
    else:
        objects_per_region[i] = [j for j in range(primary_object_indices[i], len(primary_objects) + len(secondary_objects))]
print(objects_per_region)

[[0, 1, 2, 3], [4, 5, 6], [7, 8, 9, 10]]


In [145]:
prompt7 = " Given the primary objects: " +str(primary_objects) + ", and their indices: " + str(primary_object_indices)
prompt7 += ", as well as the secondary objects " + lang5.choices[0].message.content + " and their indices: " + str(secondary_object_indices)
prompt7 += " and their individual constraints: " + lang6.choices[0].message.content
prompt7 += ", and these are the region names: " + region_names
prompt7 += " . Using the script attached, transform each constraint into function calls that will ensure that the secondary objects are set up correctly. Match each constraint"
prompt7 += " to the closest function in the script file by using the docstrings. If you cannot find a function that matches the constraint, leave it out."
prompt7 += "Here is the first script: " + file_contents3
prompt7 += "For the second script, I want you to transform the pairwise constraints into function calls. Here is the second script: " + file_contents4
prompt7 += " and here is the list of pairwise constraints: " + lang7.choices[0].message.content
prompt7 += ". I want the output to begin with:"
prompt7 += "def optimize_secondary_objects(positions, room): \n output = 0\n "
prompt7 += "followed by each constraint function call added to the output, (each line should begin with 'output +=')."
prompt7 += " Ensure to use ind_in_bounds, and ind_no_overlap. Also ensure to use ind_in_region for each object."
prompt7 += "No extra text please, only the functions and the output. Do not define ANY functions, only call them. "

# Define the request payload.
data = {
    "model": "gpt-4o",
    "messages": [
            {"role": "user", "content": prompt7}
            ]
    }
    
# Make the API call
response7 = requests.post(url, headers=headers, json=data)  
# Check the response
if response7.status_code == 200:

    if "```" in response7.json()['choices'][0]['message']['content']:
        list_objects = (response7.json()['choices'][0]['message']['content']).split("\n")[1:-1]
        response7_1 = ""
        for i in range(len(list_objects)):
            response7_1 += list_objects[i] + "\n"
    else: 
        
        response7_1 = response7.json()['choices'][0]['message']['content']
    
    indent1 = response7_1.split("output")[0].split("\n")[1]
    indent2 = primary_accessible_constraints.split("output")[0].split("\n")[1]
    response7 = "def optimize_secondary_objects(positions, room):\n" + indent1 + "output = 0\n"
    response7 += indent1 + (indent1).join(primary_accessible_constraints.split(indent2)) + "\n" + ("\n").join((response7_1.split("return")[0]).split("\n")[2:]) + "\n" + indent1 + "return output \n"
    print(response7)

else:
    print('Failed to get a response')
    print(response7.text)



def optimize_secondary_objects(positions, room):
    output = 0
    
    output += ind_accessible(positions, room, 0, ['left', 'right'])
    output += ind_accessible(positions, room, 4, ['front'])
    output += ind_next_to_wall(positions, room, 1, 'back')
    output += ind_close_to_fixed_object(positions, room, 1, 'socket', 'back', 0.5)
    output += ind_next_to_wall(positions, room, 2, 'back')
    output += ind_close_to_fixed_object(positions, room, 2, 'socket', 'back', 0.5)
    output += ind_next_to_wall(positions, room, 3, 'back')
    output += ind_accessible(positions, room, 3, ['front'])
    output += ind_next_to_wall(positions, room, 5, 'back')
    output += ind_away_from_fixed_object(positions, room, 5, 'window', 2.0)
    output += ind_accessible(positions, room, 5, ['front'])
    output += ind_not_against_wall(positions, room, 6, None, 0.2)
    output += ind_next_to_wall(positions, room, 8, 'back')
    output += ind_next_to_wall(positions, room, 9, 'back')
    output += ind_acc

In [201]:
secondary_functions = ['' for i in range(num_regions)]
lines = response7.split("\n")
for i in range(num_regions - 1, -1, -1):
    function = ("\n").join(lines[:2])

    for obj_index in objects_per_region[i]:
        lines_to_remove = []
        for line in lines:
            sub_sections = line.split(room_name + ", ")

            if "ind_accessible" in line and str(obj_index) in line:
                function += "\n" + line

            for j in sub_sections: 
                if str(obj_index) + "," in j or str(obj_index) + ")" in j:
                    function += "\n" + line
                    lines_to_remove += [line]
                    break 
    
        for line in lines_to_remove:
            if line in lines: 
                lines.remove(line)
    for line in lines: 
        if "ind_no_overlap" in line:
            function += "\n" + line
        if "ind_in_bounds" in line:
            function += "\n" + line
        if "return" in line: 
            function += "\n" + line

            
    secondary_functions[i] = remove_identical_lines(function)

In [202]:
print(secondary_functions[0])

def optimize_secondary_objects(positions, room):
    output = 0
    output += ind_accessible(positions, room, 0, ['left', 'right'])
    output += p_next_to(positions, room, 1, 0, 'left', 'left')
    output += p_next_to(positions, room, 2, 0, 'right', 'right')
    output += ind_next_to_wall(positions, room, 1, 'back')
    output += ind_in_region(positions, room, 1, 'Sleeping Area')
    output += ind_next_to_wall(positions, room, 2, 'back')
    output += ind_in_region(positions, room, 2, 'Sleeping Area')
    output += ind_next_to_wall(positions, room, 3, 'back')
    output += ind_accessible(positions, room, 3, ['front'])
    output += ind_in_region(positions, room, 3, 'Dressing Area')
    output += ind_in_bounds(positions, room)
    output += ind_no_overlap(positions, room)
    return output 


In [203]:
secondary_accessible_constraints = [[] for i in range(num_secondary_objects)]
for region in range(num_regions):
    for i in secondary_functions[region].split("\n"):
        if "ind_accessible" in i:
            secondary_accessible_constraints[region].append(i)

    secondary_accessible_constraints[region] = "\n" +("\n").join(secondary_accessible_constraints[region])

print(secondary_accessible_constraints[0])
print(secondary_accessible_constraints[1])
print(secondary_accessible_constraints[2])


    output += ind_accessible(positions, room, 0, ['left', 'right'])
    output += ind_accessible(positions, room, 3, ['front'])

    output += ind_accessible(positions, room, 4, ['front'])
    output += ind_accessible(positions, room, 5, ['front'])

    output += ind_accessible(positions, room, 9, ['front'])


# Run the code !

In [240]:
print(primary_functions[0])

def optimize_primary_objects(positions, room):
    output = 0
    output += ind_next_to_wall(positions, room, 0, 'back')
    output += ind_away_from_fixed_object(positions, room, 0, 'window')
    output += ind_away_from_fixed_object(positions, room, 0, 'door')
    output += ind_close_to_fixed_object(positions, room, 0, 'socket')
    output += ind_accessible(positions, room, 0, ['left', 'right'])
    output += ind_away_from_fixed_object(positions, room, 0, 'wardrobe')
    output += p_away_from(positions, room, 0, 8)
    output += p_aligned(positions, room, 0, 8, None, 2.0)
    output += ind_in_bounds(positions, room)
    output += ind_no_overlap(positions, room)
    return output


In [239]:
## Run the room setup 
global_context = globals().copy()
local_context = {}
exec(response3, global_context, local_context)
exec(primary_functions[0], global_context, local_context) # define the very first optimizaton function 
## Now want to add in the first primary object
exec(object_creations[0]) # add in the first object
print(object_creations[0])

options = {'maxiter': 300, 'ftol': 1e-6}
iters = 0
min_fun = np.Inf
room = local_context[room_name]
func = local_context['optimize_primary_objects']
while min_fun > 1e-2 and iters < 15:
    positions = np.zeros(3*len(room.moving_objects))
    for i in range(len(room.moving_objects)):
        positions[3*i] = np.random.uniform(0, room.width)
        positions[3*i + 1] = np.random.uniform(0, room.length)
        positions[3*i + 2] = np.random.uniform(0, 2*np.pi) 
    res = minimize(func, positions, args = (room), method = 'SLSQP', options = options) 
    iters += 1
    if res.fun < min_fun:
        if not ind_no_overlap(res.x, room) > 0.4 and not ind_in_bounds(res.x, room) > 0.4:
            min_fun = res.fun
            best_res = res
            print("New best result found. Cost: ", min_fun)

    
for i in range(len(room.moving_objects)): 
    room.moving_objects[i].position = best_res.x[3*i:3*i + 3] 
room.draw() 

room.regions[0].x, room.regions[0].y = room.moving_objects[0].position[0], room.moving_objects[0].position[1]
prev_cost = min_fun 

room.fm_indices += [0]
room.draw()

create_moving_object(local_context[room_name], "bed", 1.8, 2.0, "Sleeping Area", 0)


ValueError: not enough values to unpack (expected 3, got 0)